<a href="https://colab.research.google.com/github/thisisphume/ETF_Analysis_Testing/blob/main/utilities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Back up
price_data = web.get_data_yahoo(etf_list,start = '2014-01-01',end = '2018-05-31')['Adj Close']

### Retrieve stock data
eft_close = stock_price_retrieval(etf_list, '1356998400',  '1577836800', "1wk", "Adj Close") #2013-01-01 - 2020-01-01
print ("Number of Missing data")
print (len(eft_close.isna().any()[eft_close.isna().any() == True]))

In [ ]:
### Retrieving the stock price
def stock_price_retrieval(stock_list, start, end, interval, whichprice ):
  '''
  https://finance.yahoo.com/quote/GE/history?period1=0&period2=1591920000&interval=1d&filter=history&frequency=1d
  stock_list: (list) list of stock
  start     : (str) check the website i.e. "1267142400"
  end       : (str) check the website i.e. "1546300800"
  interval  : (str) "1d", "1wk"
  whichprice: (str) "Adj Close", "High", "Low"
  '''

  ### Setting up the dataframe. We don't care about the GE data but we want the index-date
  url = 'https://query1.finance.yahoo.com/v7/finance/download/'\
          '{}?period1={}&period2={}&interval={}&events=history'.format("GE", start, end, interval)
  response = requests.get(url)
  df = pd.read_csv(io.StringIO(response.text), parse_dates=['Date'])


  ### Bulk retrieval
  df_main = df[['Date']]

  for stock in stock_list:

      try:
        url = 'https://query1.finance.yahoo.com/v7/finance/download/'\
            '{}?period1={}&period2={}&interval={}&events=history'.format(stock, start, end, interval)
        response = requests.get(url)
        df = pd.read_csv(io.StringIO(response.text), parse_dates=['Date'])
        
        
        temp = df[['Date',whichprice]] 
        temp.columns = ['Date', stock]
        df_main = pd.merge(df_main, temp, how = 'left' , on = 'Date')  
      except:
        print ("ERROR: ", stock)

  ### Assign index as date
  df_main.index = df_main.Date
  df_main = df_main.drop(columns=['Date'])

  return df_main